<a href="https://colab.research.google.com/github/MegRoy97/FakeNewsDetection/blob/master/FND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [5]:
data=pd.read_csv("../data.csv")
data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [6]:
data['News']=data['Headline']+' '+data['Body']
data.head()

,URLs,Headline,Body,Label,News
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,Four ways Bob Corker skewered Donald Trump Ima...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,Linklater's war veteran comedy speaks to moder...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,Jason Aldean opens 'SNL' with Vegas tribute Co...


In [7]:
data=data.drop(columns="Headline")
data.head()
data=data.drop(columns="Body")
data.head()

,URLs,Label,News
0,http://www.bbc.com/news/world-us-canada-414191...,1,Four ways Bob Corker skewered Donald Trump Ima...
1,https://www.reuters.com/article/us-filmfestiva...,1,Linklater's war veteran comedy speaks to moder...
2,https://www.nytimes.com/2017/10/09/us/politics...,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,https://www.reuters.com/article/us-mexico-oil-...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,1,Jason Aldean opens 'SNL' with Vegas tribute Co...


In [9]:
nltk.download('stopwords')
nltk_stopwords= stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def clean_column(dframe, columnToClean, newColumn):
    df_copy = dframe.copy()
    df_copy['copcol'] = df_copy[columnToClean]
    df_copy['copcol'] = df_copy['copcol'].str.lower()
    newcolumn = []
    for label in df_copy.index:
        row = df_copy.loc[label, :]['copcol']
        clean = re.split(r'[^A-Za-z]+', row)
        clean = [x for x in clean if x not in nltk_stopwords]
        clean = [x for x in clean if len(x) != 1]
        clean = " ".join(clean)
        clean = clean.strip()
        newcolumn.append(clean)
    df_copy[newColumn] = newcolumn
    del df_copy['copcol']
    return df_copy

def filtered(dframe, col):
    dframe[col] = dframe[col].apply(lambda x: x.replace(" ur ", " your "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" u ", " you "))
    dframe[col] = dframe[col].apply(lambda x: x.replace("&gt;", "greater than"))
    dframe[col] = dframe[col].apply(lambda x: x.replace("&lt;", "less than"))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" jan ", " january "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" feb ", " february "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" mar ", " march "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" apr ", " april "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" jun ", " june "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" jul ", " july "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" aug ", " august "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" sept ", " september "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" oct ", " october "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" nov ", " november "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" dec ", " december "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" hrs ", " hours "))
    dframe[col] = dframe[col].apply(lambda x: x.replace(" btw ", " by the way "))
    return dframe

In [11]:
df1=data.copy()
df1.head()

,URLs,Label,News
0,http://www.bbc.com/news/world-us-canada-414191...,1,Four ways Bob Corker skewered Donald Trump Ima...
1,https://www.reuters.com/article/us-filmfestiva...,1,Linklater's war veteran comedy speaks to moder...
2,https://www.nytimes.com/2017/10/09/us/politics...,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,https://www.reuters.com/article/us-mexico-oil-...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,1,Jason Aldean opens 'SNL' with Vegas tribute Co...


In [12]:
print(data.isna().any(axis=1))

0       False
1       False
2       False
3       False
4       False
        ...  
4004    False
4005    False
4006     True
4007    False
4008    False
Length: 4009, dtype: bool


In [0]:
data=data[data.News.notnull()]

In [14]:
data[data['News'].isnull()]

,URLs,Label,News


In [0]:
df1=data.copy()
df1 = clean_column(df1, 'News', 'clean_News')

In [16]:
df1['clean_News']

0       four ways bob corker skewered donald trump ima...
1       linklater war veteran comedy speaks modern ame...
2       trump fight corker jeopardizes legislative age...
3       egypt cheiron wins tie pemex mexican onshore o...
4       jason aldean opens snl vegas tribute country s...
                              ...                        
4003    cnn globalist exposed steve quayle alex jones ...
4004    trends watch trends watch readers think story ...
4005    trump jr soon give minute speech trump jr soon...
4007    china accept overseas trial data bid speed dru...
4008    vice president mike pence leaves nfl game anti...
Name: clean_News, Length: 3988, dtype: object

In [17]:
df1 = filtered(df1, 'clean_News')
df1['clean_News']

0       four ways bob corker skewered donald trump ima...
1       linklater war veteran comedy speaks modern ame...
2       trump fight corker jeopardizes legislative age...
3       egypt cheiron wins tie pemex mexican onshore o...
4       jason aldean opens snl vegas tribute country s...
                              ...                        
4003    cnn globalist exposed steve quayle alex jones ...
4004    trends watch trends watch readers think story ...
4005    trump jr soon give minute speech trump jr soon...
4007    china accept overseas trial data bid speed dru...
4008    vice president mike pence leaves nfl game anti...
Name: clean_News, Length: 3988, dtype: object

In [0]:
df1.to_pickle("cleanedData.pkl")

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
df2=df1.copy()
df2['new_col']=df1['clean_News']
lemm_col=[]
for label1 in df2.index:
    row = df2.loc[label1, :]['new_col']
    clean = nltk.word_tokenize(row)
    clean=' '.join([lemmatizer.lemmatize(w) for w in clean])
    lemm_col.append(clean)
df2['new_col']=lemm_col

In [25]:
df2.head()

,URLs,Label,News,clean_News,new_col
0,http://www.bbc.com/news/world-us-canada-414191...,1,Four ways Bob Corker skewered Donald Trump Ima...,four ways bob corker skewered donald trump ima...,four way bob corker skewered donald trump imag...
1,https://www.reuters.com/article/us-filmfestiva...,1,Linklater's war veteran comedy speaks to moder...,linklater war veteran comedy speaks modern ame...,linklater war veteran comedy speaks modern ame...
2,https://www.nytimes.com/2017/10/09/us/politics...,1,Trump’s Fight With Corker Jeopardizes His Legi...,trump fight corker jeopardizes legislative age...,trump fight corker jeopardizes legislative age...
3,https://www.reuters.com/article/us-mexico-oil-...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...,egypt cheiron wins tie pemex mexican onshore o...,egypt cheiron win tie pemex mexican onshore oi...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,1,Jason Aldean opens 'SNL' with Vegas tribute Co...,jason aldean opens snl vegas tribute country s...,jason aldean open snl vega tribute country sin...


In [0]:
df2.to_pickle("df2.pkl")

In [27]:
tfidf = TfidfVectorizer(ngram_range=(2,2), max_df= 0.85, min_df= 0.01)
X=df2['new_col']
Y=df2['Label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 50)
X_train_tfidf=tfidf.fit_transform(X_train)
num=len(tfidf.get_feature_names())
num

649

In [28]:
tfidf = TfidfVectorizer(ngram_range=(2,2), max_df= 0.85, min_df= 0.01, max_features=num)
X_test_tfidf=tfidf.fit_transform(X_test)
num=len(tfidf.get_feature_names())
num

649

In [29]:
#DECISION TREE USING GINI INDEX
clf_gini = DecisionTreeClassifier(criterion = "gini", 
            random_state = 85,max_depth=10, min_samples_leaf=2) 
clf_gini.fit(X_train_tfidf, y_train) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=85, splitter='best')

In [30]:
y_pred=clf_gini.predict(X_test_tfidf)
print(y_pred)

[1 1 1 ... 0 1 1]


In [31]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score*100

77.61069340016708

In [32]:
from sklearn import model_selection, naive_bayes, svm

#NAIVE BAYES CLASSIFIER
nbc=naive_bayes.MultinomialNB()
nbc.fit(X_train_tfidf, y_train)
y_pred_nbc=nbc.predict(X_test_tfidf)
y_pred_nbc

array([0, 1, 0, ..., 0, 1, 1])

In [33]:
score=accuracy_score(y_test,y_pred_nbc)
score*100

70.59314954051797

In [36]:
#SVM
svmc=svm.SVC(C=10,kernel='linear')
svmc.fit(X_train_tfidf,y_train)
y_pred_svm=svmc.predict(X_test_tfidf)
y_pred_svm

array([1, 1, 1, ..., 0, 0, 1])

In [37]:
score=accuracy_score(y_test,y_pred_svm)
score*100

73.26649958228906